In [1]:
%matplotlib inline
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

import Vector_Similarity
import rank_document

from read_folds import FoldsTraining
from TF_IDF_Improved import TFIDFImproved

# TRAIN_SET_PATH = "20ng-no-stop.txt"
# TRAIN_SET_PATH = "r52-all-terms.txt"
TRAIN_SET_PATH = "dataset/r8-no-stop.txt"

GLOVE_6B_50D_PATH = "dataset/glove.6B.50d.txt"
GLOVE_840B_300D_PATH = "dataset/glove.6B.300d.txt"
encoding="utf-8"

C:\Users\motos\machine_learning\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:


folds_training = FoldsTraining(nb_folds=2)
# relevance_list = [item for sublist in folds_training.true_relevance_stemmed for item in sublist]
# relevance_list = folds_training.true_relevance_stemmed[0]
queries_list_train = folds_training.queries[0]
queries_list_test = folds_training.queries[1]
paragraphs_dict = folds_training.paragraphs_dict

print(queries_list_train[20])


tfidf = TFIDFImproved(paragraphs_dict)
tfidf_ranked = tfidf.ranked_with_vector(queries_list_test, 20)

Bad para:  4


Bad para:  4
('enwiki:Heavy%20water/Effect%20on%20biological%20systems/Toxicity%20in%20humans', 'Heavy water Effect on biological systems Toxicity in humans', ['heavi', 'water', 'effect', 'biolog', 'system', 'toxic', 'human'])


In [4]:
X, y = [], []
y_mapping = {}
y_mapping_index = 0
i = 0

for query in queries_list_train:
    X.append(query[2])

for par_id, par_text in paragraphs_dict.items():
    y.append(par_text)

# for query_relevance in relevance_list:
#     # if i == 0:
#         # print(query_relevance)
#     query = query_relevance[0]
#     relevance = query_relevance[1]
#     for par_id, rel in relevance.items():
#         # if i == 0:
#             # print(query, par_id, paragraphs_dict[par_id])
#             # i += 1
#         X.append(query)
#         y.append(paragraphs_dict[par_id])


X, y = np.array(X), np.array(y)
print(X[0], y[0])
print("total x =", len(X), "y =", len(y))


['chocol', 'etymolog'] ['peopl', 'schizophrenia', 'tend', 'also', 'gi', 'problem', '2015', 'studi', 'carri', 'compar', 'gut', 'flora', 'peopl', 'schizophrenia', 'healthi', 'peopl', 'research', 'caus', 'schizophrenialik', 'symptom', 'mice', 'give', 'phencyclidin', 'pcp', 'found', 'chang', 'gut', 'flora', 'treat', 'mice', 'compar', 'untreat', 'mice']
total x = 388 y = 1815


In [5]:
# reading glove files, this may take a while
# we're reading line by line and only saving vectors
# that correspond to words from our training set
# if you wan't to play around with the vectors and have 
# enough RAM - remove the 'if' line and load everything

import struct 
from stemming.porter2 import stem

glove_small = {}
# all_words = set(w for words in X for w in words)
with open(GLOVE_6B_50D_PATH, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding)
        # if (word in all_words):
        nums=np.array(parts[1:], dtype=np.float32)
        glove_small[stem(word)] = nums

            
# glove_big = {}
# with open(GLOVE_840B_300D_PATH, "rb") as infile:
#     for line in infile:
#         parts = line.split()
#         word = parts[0].decode(encoding)
#         nums=np.array(parts[1:], dtype=np.float32)
#         glove_small[word] = nums
            

In [6]:

# and a tf-idf version of the same
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        if len(word2vec) > 0:
            self.dim = len(word2vec[next(iter(glove_small))])
        else:
            self.dim = 0

    def fit(self, X, y):

        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] * self.word2weight[w]
                     for w in words if w in self.word2vec] or
                    [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [7]:
tfidf_vectorizer = TfidfEmbeddingVectorizer(glove_small).fit(np.append(X, y), y=None)
# tfidf_y = TfidfEmbeddingVectorizer(glove_small).fit(y, y=None)
# x_vec = tfidf.transform(X)
# y_vec = tfidf.transform(y)

# x_vec_X = tfidf.transform(X)
# y_vec_X = tfidf.transform(y)


In [8]:
stop = 0
for k,v in tfidf_ranked.items():
    if stop == 0:
        print(k)
        print(v[0])
        print(v[1])
    stop+=1

In [9]:
import operator

topn = 10

# query = queries_list_test[10]
final_scores_tsss = {}
final_scores_cosine = {}
stop = 0
progress = 0

stop = 0
for query_id, ranking in tfidf_ranked.items():
    q = ranking[0]
    
    # if stop == 0:
    #     print(query_id)
#         print(v[0])
#         print(v[1])
#     stop+=1
# 
# 
# for query in queries_list_test:
#     q = query[2]
    # print(queries_list_test[10])
    # print(q)

    scores_tsss = {}
    scores_cosine = {}
    q_vec = tfidf_vectorizer.transform([q])[0]
    for r in ranking[1]:
        par_id = r[0]
        score = r[1]
        # if stop == 0:
        #     print(par_id, paragraphs_dict[par_id])
        
        par_text_vec = tfidf_vectorizer.transform([paragraphs_dict[par_id]])[0]
        scores_tsss[par_id] = Vector_Similarity.TS_SS(q_vec, par_text_vec)
        scores_cosine[par_id] = Vector_Similarity.Cosine(q_vec, par_text_vec)
    # stop+=1
    sorted_scores_tsss = sorted(scores_tsss.items(), key=operator.itemgetter(1), reverse=True)[:topn]
    sorted_scores_cosine = sorted(scores_cosine.items(), key=operator.itemgetter(1), reverse=True)[:topn]

    for tsss in sorted_scores_tsss:
        if final_scores_tsss.get(query_id) is None:
            final_scores_tsss[query_id] = []
        final_scores_tsss[query_id].append([tsss[0], tsss[1]])

    for cosine in sorted_scores_tsss:
        if final_scores_cosine.get(query_id) is None:
            final_scores_cosine[query_id] = []
        final_scores_cosine[query_id].append([cosine[0], cosine[1]])

    # print()
    # print(final_scores_cosine)
    

    progress += 1
    print("progress:", "%.3f" % round(progress/len(queries_list_test), 3))
    # break



progress: 0.005
progress: 0.009


progress: 0.014
progress: 0.019
progress: 0.023


progress: 0.028
progress: 0.033


progress: 0.037
progress: 0.042


progress: 0.047
progress: 0.051


...

progress: 0.939
progress: 0.944


progress: 0.949
progress: 0.953
progress: 0.958


progress: 0.963
progress: 0.967
progress: 0.972


progress: 0.977
progress: 0.981
progress: 0.986


progress: 0.991
progress: 0.995
progress: 1.000


In [12]:
for k,v in tfidf_ranked.items():
    print(k,v)

enwiki:Heavy%20water/Explanation [('266ed623b35a182f2a82d9f52d1cc99d24f13e0f', 1.0029704882642545), ('17146b1d01124c2b47131439f34624265fd80a58', 0.984573145512143), ('6d7391c271240d8a44b0234001fb9e48afc033d8', 0.9333399138194606), ('bcdea2ca3e494239678608558eec0f4913efea4f', 0.9186519168263868), ('765f3bbb2d1e283dee14476f16467bfe0e1180a0', 0.9081060719264888), ('eb6e40ac08ba9e8ca3173e29c0e0d5990033cac4', 0.888263935249987), ('83e7c5ab4680820b95e0c4dfba823c3c6551d1b1', 0.8776201448448043), ('74237e271d9021c7b09653bc72095c372ffb8a59', 0.844363906977424), ('08cf162d20d9bbcaafa0bd97ca2a47fb2a6ae4bf', 0.8415164069152486), ('78ecbe2fa54c2333e57bbe64f8bd3f2152f653af', 0.796450977266575), ('1b4a3b21c2b13d3271c1e887b30cf5f0cb617775', 0.7712964002765461), ('6df310dc50d982184f15c223b672c53d8e7a383b', 0.7635014028802076), ('4bf4d7700df8abec11f0bebf71aefcafbacfc1ce', 0.7281437445702327), ('8c5a17a9cdc3d593689a6c9d816862eacf84bf8d', 0.676553739838552), ('a0468d93ac24b9938c4fc26cfec40aa77b6a9759', 0.

In [10]:
from trec_car.format_runs import *
stop = 0
output_entries_tsss = []
for query_id, paragraphs in final_scores_tsss.items():     
    rank = 1
    for paragraph_score in paragraphs:        
        entry = RankingEntry(query_id, paragraph_score[0], rank, paragraph_score[1])
        output_entries_tsss.append(entry)
        rank += 1
        
print(len(output_entries_tsss))

2140


In [11]:
output_entries_cosine = []
for query_id, paragraphs in final_scores_cosine.items():     
    rank = 1
    for paragraph_score in paragraphs:        
        entry = RankingEntry(query_id, paragraph_score[0], rank, paragraph_score[1])
        output_entries_cosine.append(entry)
        rank += 1

In [12]:
def save_scores_to_file(output_entries, filename="test.out"):
    with open(filename, mode='w', encoding='UTF-8') as f:
        writer = f
        temp_list = []
        for entry in output_entries:
            temp_list.append(entry)
        format_run(writer, temp_list, exp_name='test')
        f.close()

In [13]:
o = output_entries_tsss[11]
print(o.query_id, o.paragraph_id, o.rank, o.score)
save_scores_to_file(output_entries_tsss, filename="word2vec_tsss_stem_pretfidf_top10.out")
save_scores_to_file(output_entries_cosine, filename="word2vec_cosine_stem_pretfidf_top10.out")

enwiki:Wedding%20cake/History/Pre-18th%20century%20history 95f5690bbab0236bf164a06abef3645158dad1fc 2 135.7909584702463
